<a href="https://colab.research.google.com/github/2303A51529/NLP-LAB/blob/main/NLP_F_12_9_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

df = pd.read_csv("/content/train.csv")
print(df.head())

texts = df["text"].values
labels = df["target"].values

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    return " ".join(tokens)

df["clean_text"] = df["text"].apply(clean_text)

X_train, X_val, y_train, y_val = train_test_split(df["clean_text"], labels, test_size=0.2, random_state=42)

count_vec = CountVectorizer()
X_train_count = count_vec.fit_transform(X_train)
X_val_count = count_vec.transform(X_val)

tfidf_vec = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vec.fit_transform(X_train)
X_val_tfidf = tfidf_vec.transform(X_val)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_val_tfidf)

svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)
y_pred_svm = svm.predict(X_val_tfidf)

print("\n--- Logistic Regression (TF-IDF) ---")
print(classification_report(y_val, y_pred_lr))

print("\n--- SVM (TF-IDF) ---")
print(classification_report(y_val, y_pred_svm))

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["clean_text"])
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

max_len = 30
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post")
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding="post")

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

mlp = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])
mlp.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
mlp.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_mlp = (mlp.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- MLP (Embeddings) ---")
print(classification_report(y_val, y_pred_mlp))

cnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation="relu"),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])
cnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
cnn.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_cnn = (cnn.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- CNN (Embeddings) ---")
print(classification_report(y_val, y_pred_cnn))

lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(128, dropout=0.3, recurrent_dropout=0.3),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])
lstm.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
lstm.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_lstm = (lstm.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- LSTM (Embeddings) ---")
print(classification_report(y_val, y_pred_lstm))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  

--- Logistic Regression (TF-IDF) ---
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       874
           1       0.82      0.68      0.74       649

    accuracy                           0.80      1523
   macro avg       0.80      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523


--- SVM (TF-IDF) ---
              precision    recall  f1-score   support

           0       0.79

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5550 - loss: 0.6833 - val_accuracy: 0.5982 - val_loss: 0.6504
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6915 - loss: 0.5991 - val_accuracy: 0.7991 - val_loss: 0.4621
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8358 - loss: 0.3904 - val_accuracy: 0.7945 - val_loss: 0.4539
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8775 - loss: 0.2910 - val_accuracy: 0.7892 - val_loss: 0.4710
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9081 - loss: 0.2409 - val_accuracy: 0.7262 - val_loss: 0.5874
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

--- MLP (Embeddings) ---
              precision    recall  f1-score   support

           0       0.82      0.66      0.74       874
           1       0.64      0.81      0.72       649

    accuracy                           0.73      1523
   macro avg       0.73      0.74      0.73      1523
weighted avg       0.75      0.73

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


191/191 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.6335 - loss: 0.6369 - val_accuracy: 0.7925 - val_loss: 0.4697
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8807 - loss: 0.3137 - val_accuracy: 0.7682 - val_loss: 0.5042
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9555 - loss: 0.1470 - val_accuracy: 0.7413 - val_loss: 0.6671
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9691 - loss: 0.0885 - val_accuracy: 0.7347 - val_loss: 0.8592
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9750 - loss: 0.0554 - val_accuracy: 0.7393 - val_loss: 0.9222
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

--- CNN (Embeddings) ---
              precision    recall  f1-score   support

           0       0.78      0.76      0.77       874
           1       0.69      0.71      0.70       649

    accuracy                           0.74      1523
   macro avg       0.73      0.74      0.73      1523
weighted avg       0.74      0.74

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


191/191 ━━━━━━━━━━━━━━━━━━━━ 25s 94ms/step - accuracy: 0.5626 - loss: 0.6858 - val_accuracy: 0.7498 - val_loss: 0.5982
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 18s 82ms/step - accuracy: 0.7671 - loss: 0.5285 - val_accuracy: 0.6907 - val_loss: 0.5985
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 16s 83ms/step - accuracy: 0.8026 - loss: 0.4856 - val_accuracy: 0.7150 - val_loss: 0.6335
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.8145 - loss: 0.4645 - val_accuracy: 0.5745 - val_loss: 0.6426
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 16s 82ms/step - accuracy: 0.7610 - loss: 0.4759 - val_accuracy: 0.5430 - val_loss: 0.6637
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step

--- LSTM (Embeddings) ---
              precision    recall  f1-score   support

           0       0.84      0.25      0.39       874
           1       0.48      0.94      0.64       649

    accuracy                           0.54      1523
   macro avg       0.66      0.59      0.51      1523
weighted avg       0.69   